# Tâche 2 - Comme le disait le proverbe - les bons mots pour le dire

L'objectif de cette tâche est de compléter des proverbes à l'aide de modèles de langue N-grammes en insérant des mots aux bons endroits dans un texte masqué. Il s'agit d'une tâche de type *cloze test* qui consiste à choisir le meilleur mot à insérer dans un texte en fonction de son contexte. 

Voir l'énoncé du travail #1 pour une description plus détaillée de cette tâche. 

Fichiers:
- *proverbes.txt*: il contient plus de 3000 proverbes, un par ligne de texte. Pour l'entraînement des modèles de langues N-grammes. 
- *test_proverbes_v1.json*: il contient des proverbes masqués, les mots à insérer et la bonne formulation du proverbe. À utiliser pour évaluer la capacité des modèles à mettre les mots aux bons endroits. 

Consignes: 
- Utilisez NLTK pour construire les modèles de langue.
- Utilisez des expressions régulières (une seule ou plusieurs) pour remplacer les * et les ** par des mots. 
- Utilisez NLTK pour faire la tokenisation des proverbes. 
- N'oubliez pas de faire le *padding* des proverbes avec des symboles de début \<BOS\> et de fin \<EOS\>.
- Ne pas modifier les fonctions *load_proverbs* et *load_tests*.
- Ne pas modifier la signature de la fonction *replace_stars_with_words*.
- Utilisez la variable *models* pour conserver les modèles après entraînement. 
- Ne pas modifier la signature de la fonction *train_models*.
- Ne pas modifier la signature de la fonction *fill_masked_proverb*. 
- Des modifications aux signatures pourraient entraîner des déductions dans la correction. 
- Vous pouvez ajouter des cellules au notebook et toutes les fonctions utilitaires que vous voulez. 

## Section 1 - Lecture des fichiers de données (proverbes et tests)

In [26]:
import json

# Ne pas modifier le chemin de ces 2 fichiers pour faciliter notre correction
proverbs_fn = "./data/proverbes.txt"    
test_v1_fn = './data/test_proverbes_v1.json'

def load_proverbs(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_lines = f.readlines()
    return [x.strip() for x in raw_lines]


def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [27]:
proverbs = load_proverbs(proverbs_fn)

In [28]:
print("Nombre de proverbes pour l'entraînement: {}".format(len(proverbs)))
print("Un exemple de proverbe: " + proverbs[5])

Nombre de proverbes pour l'entraînement: 3108
Un exemple de proverbe: accord vaut mieux qu’argent


In [29]:
tests = load_tests(test_v1_fn)

In [30]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests)
df

,Masked,Word_list,Proverb
0,a beau * qui ** de loin,"[vient, mentir]",a beau mentir qui vient de loin
1,a * mentir qui vient de **,"[beau, loin]",a beau mentir qui vient de loin
2,l’* fait le **,"[larron, occasion]",l’occasion fait le larron
3,"*-toi, le ciel t’**","[aidera, aide]","aide-toi, le ciel t’aidera"
4,"année de *, ** de blé","[année, gelée]","année de gelée, année de blé"
5,"après la *, le ** temps","[beau, pluie]","après la pluie, le beau temps"
6,"aux échecs, les * sont les plus près des **","[fous, rois]","aux échecs, les fous sont les plus près des rois"
7,"ce que * veut, ** le veut","[femme, dieu]","ce que femme veut, dieu le veut"
8,bien mal * ne ** jamais,"[profite, acquis]",bien mal acquis ne profite jamais
9,bon * ne querelle pas ses **,"[outils, ouvrier]",bon ouvrier ne querelle pas ses outils


## Section 2 - Code pour substituer les masques (étoiles) par des mots

Expliquez ici comment vous procédez pour remplacer les étoiles des proverbes masqués par des mots... N'oubliez pas qu'il faut faire usage d'expressions régulières (une ou plusieurs - au choix). 



In [31]:
import re

def replace_stars_with_words(masked, word1, word2):
    """Remplace les * par word1 et word2 dans cet ordre. Retourne le proverbe complet."""
    
    match = re.sub("\*\*", word2, masked)
    res = re.sub("\*", word1, match)
        
    return res  # Retourne le proverbe avec des mots à la place des étoiles


Pour remplacer des * par word1 et ** par word2 avec des regex on a utilisé la méthode sub de regex qui remplace la première occurence d'un match de regex
dans un texte par un autre texte donnée en paramètre.
Ainsi on crée une regex détectant la chaine de caractères '**' et la remplace par word2.
Ensuite on détecte la regex détectant le caractère '*' et le remplace par word1.

## Section 3 - Construction des modèles de langue N-grammes. 

La fonction ***train_models*** prend en entrée une liste de proverbes et construit les trois modèles unigramme, bigramme et trigramme.

Les 3 modèles entraînés sont conservés dans *models*, un dictionnaire Python qui prend la forme 

<pre>
{
   1: modele_unigramme, 
   2: modele_bigramme, 
   3: modele_trigramme
}
</pre>

avec comme clé la valeur N du modèle et comme valeur le modèle construit par NLTK.

Expliquez ici comment vous procéder pour construire vos modèles avec NLTK, pour obtenir les n-grammes de mots, pour déterminer le vocabulaire, etc...



In [33]:
import nltk
#Uncomment those tree lines if package are missing
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.util import pad_sequence
from nltk import word_tokenize, bigrams, trigrams
from nltk.util import ngrams
from nltk.lm import MLE
from nltk.lm.models import Laplace

# autres librairies à importer pour la partie sur les N-grammes?

BOS = '<BOS>'  # Jeton de début de proverbe
EOS = '<EOS>'  # Jeton de fin de proverbe

models = {}  # les modèles entraînés - [1: modele_unigramme, 2: modele_bigramme, 3: modele_trigramme] 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...


In [34]:
def train_models(proverbs):
    """ Vous ajoutez à partir d'ici le code dont vous avez besoin
        pour construire les différents modèles N-grammes.
        Cette fonction doit construire tous les modèles en une seule passe.
        Voir les consignes de l'énoncé du travail pratique concernant les modèles à entraîner.

        Vous pouvez ajouter les fonctions/méthodes et variables que vous jugerez nécessaire.
        Merci de ne pas modifier la signature et le comportement de cette fonction (nom, arguments, sauvegarde des modèles).
    """
    
    #Create vocabulary
    
    all_unigrams = list()
    for sentence in proverbs:
        word_list = word_tokenize(sentence)
        all_unigrams = all_unigrams + word_list
    vocabulary = set(all_unigrams)
    vocabulary.add(BOS)
    vocabulary.add(EOS)
    vocabulary = list(vocabulary)
    
    #Tokenisation 
    modele_unigramme=[]
    modele_bigramme=[]
    modele_trigramme=[]
    
    for proverb in proverbs:
        tokens = word_tokenize(proverb)
        #modele_unigramme.extend(list(pad_sequence(tokens, pad_left = True, left_pad_symbol = BOS, pad_right = True, right_pad_symbol = EOS, n = 1)))
        padded_sent = list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=1))
        modele_unigramme = modele_unigramme + list(ngrams(padded_sent, n=1))  
        
        #modele_bigramme.extend(list(pad_sequence(tokens, pad_left = True, left_pad_symbol = BOS, pad_right = True, right_pad_symbol = EOS, n = 2)))
        padded_sent = list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=2))
        modele_bigramme = modele_bigramme + list(ngrams(padded_sent, n=2))  
        
        #modele_trigramme.extend(list(pad_sequence(tokens, pad_left = True, left_pad_symbol = BOS, pad_right = True, right_pad_symbol = EOS, n = 3)))
        padded_sent = list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=3))
        modele_trigramme = modele_trigramme + list(ngrams(padded_sent, n=2)) 
    
    #Training models
    model1 = Laplace(1)
    model1.fit([modele_unigramme], vocabulary_text=vocabulary)
    model2 = Laplace(2)
    model2.fit([modele_bigramme], vocabulary_text=vocabulary)
    model3 = Laplace(3)
    model3.fit([modele_trigramme], vocabulary_text=vocabulary)  
    
    # Sauvegarde de vos modèles 
    models[1] = model1
    models[2] = model2
    models[3] = model3

train_models(proverbs)

## Section 4 - Compléter un proverbe

In [37]:
def fill_masked_proverb(masked, word_list, n=3, criteria="perplexity"):
    """ Fonction qui complète un texte à trous (des mots masqués) en ajoutant 
        les bons mots aux bons endroits (un "cloze test").

        Le paramètre criteria indique la mesure qu'on utilise 
        pour choisir le mot le plus approprié: "logprob" ou "perplexity".
        On retourne l'estimation de cette mesure sur le proverbe complet,
        c.-à-d. en utilisant tous les mots du proverbe.

        Le paramètre n désigne le modèle utilisé.
        1 - unigramme NLTK, 2 - bigramme NLTK, 3 - trigramme NLTK
        
        Cette fonction retourne la solution (le proverbe complété) et 
        la valeur de logprob ou perplexité (selon le paramètre en entrée de la fonction). 
    """

    # Votre code à partir d'ici. Vous pouvez modifier comme bon vous semble.
    
    #revoie les n token avant le caractère * 
    def get_before_star(sentence:str, n:int) -> str:
        res = []
        tokens =  word_tokenize(sentence)
        for token in tokens:
            if(token == '*'):
                break
            res.append(token)
            
        if(len(res) > n):
            for _ in range(0, len(res) - n):
                res.pop(0)
        return res
    
    #revoie les n token avant le caractère ** et apres le caractère * 
    def get_before_doublestar(sentence:str, n:int) -> str:
        res = []
        tokens =  word_tokenize(sentence)
        first_star_found = False
        for token in tokens:
            if first_star_found:
                if(token == '*'):
                    break
                res.append(token)
            else:
                if(token == '*'):
                    first_star_found = True
        if(len(res) > n):
            for _ in range(0, len(res) - n):
                res.pop(0)
        return res
    
    
    def get_ngramm_sequence(sentence:str, n:int) -> str:
        def clone(arr):
            res = []
            for i in range(0, len(arr)):
                res.append(arr[i])
            return res
          
        tokens = word_tokenize(sentence)
        res = [[tokens[i] for i in range(0, n)]]
        for _ in range(0, n):
            tokens.pop(0)
        
        i = 0
        while(len(tokens) > 0):
            tmp = clone(res[i])
            tmp.pop(0)
            tmp.append(tokens.pop(0))
            res.append(tmp)
            i += 1
            
        return res
    
    if(criteria == 'perplexity'):
        sentence1 = replace_stars_with_words(masked, word_list[0], word_list[1])
        sentence2 = replace_stars_with_words(masked, word_list[1], word_list[0])
        
        ngramm_sentence1 = get_ngramm_sequence(sentence1, n)
        perplexity1 = models[n].perplexity(ngramm_sentence1)
        
        ngramm_sentence2 = get_ngramm_sequence(sentence2, n)
        perplexity2 = models[n].perplexity(ngramm_sentence2)
        
        if(perplexity1 < perplexity2):
            return sentence1, perplexity1
        else:
            return sentence2, perplexity2
    else:
        beg = get_before_star(masked, n)
        end = get_before_doublestar(masked, n)
        probWord1 = models[n].score(word_list[0], beg)
        probWord2 = models[n].score(word_list[1], end)
        probWord3 = models[n].score(word_list[1], beg)
        probWord4 = models[n].score(word_list[0], end)
        
        prob1 = probWord1 * probWord2
        prob2 = probWord3 * probWord4
        
        if prob1 > prob2 :
            return replace_stars_with_words(masked, word_list[0], word_list[1]), prob1
        else:
            return replace_stars_with_words(masked,word_list[1],word_list[0]), prob2


masked =  "a beau * qui ** de loin"
word_list = [ "vient","mentir"]    
res, prob = fill_masked_proverb(masked, word_list, n=3, criteria="logprob")
print("Proverbe : " + res)
print("logprob : " + str(prob))

Proverbe : a beau mentir qui vient de loin
logprob : 2.1316133089749617e-07


## Section 5 - Expérimentations et analyse de vos résultats

Décrivez ici les résultats obtenus et présentez l'évaluation obtenue sur le fichier de test. Vous pouvez ajouter le nombre de cellules que vous souhaitez. 

In [41]:
#test
tests = load_tests(test_v1_fn)

def test_model(n:int, criteria:str):
    nb_error = 0
    nb_good = 0
    for test in tests:
        masked = test['Masked']
        word_list = test['Word_list'] 
        res, _ = fill_masked_proverb(masked, word_list, n, criteria)
        if(res == test['Proverb']):
            nb_good += 1
        else:
            nb_error += 1
    return nb_error, nb_good
        
(err, ok) = test_model(1, "logprob")
print("Modèle unigramme avec logprobabilité, précision : " + str(float(ok) / float(err + ok)))
(err, ok) = test_model(1, "perplexity")
print("Modèle unigramme avec perplexité, précision : " + str(float(ok) / float(err + ok)))
(err, ok) = test_model(2, "logprob")
print("Modèle bigramme avec logprobabilité, précision : " + str(float(ok) / float(err + ok)))
(err, ok) = test_model(2, "perplexity")
print("Modèle bigramme avec perplexité, précision : " + str(float(ok) / float(err + ok)))
(err, ok) = test_model(3, "logprob")
print("Modèle trigramme avec logprobabilité, précision : " + str(float(ok) / float(err + ok)))
(err, ok) = test_model(3, "perplexity")
print("Modèle trigramme avec perplexité, précision : " + str(float(ok) / float(err + ok)))


Modèle unigramme avec logprobabilité, précision : 0.41935483870967744
Modèle unigramme avec perplexité, précision : 0.41935483870967744
Modèle bigramme avec logprobabilité, précision : 0.6451612903225806
Modèle bigramme avec perplexité, précision : 0.967741935483871
Modèle trigramme avec logprobabilité, précision : 0.6451612903225806
Modèle trigramme avec perplexité, précision : 0.45161290322580644


## Section 6 - Partie réservée pour faire nos tests lors de la correction

Merci de ne pas modifier ni retirer cette section du notebook ! 